In [1]:
import os, ast, json, uuid, re
import numpy as np
import pandas as pd
import geopandas

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
print(pio.renderers)

Renderers configuration
-----------------------
    Default renderer: 'browser'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']



In [2]:
pd.set_option('display.max_columns', None)
sns.set_theme()

In [2]:
os.getcwd()

'c:\\users\\user\\root_dir\\src\\nct_distr'

In [3]:
# all path for workplace directory
csv_path = "C:/Users/user/root_dir/data/nct-distr/dist-report-v3.csv"
zipcode_json_path = "C:/Users/user/root_dir/data/nct-distr/zipcodeall.json"
geo_distric_path = "C:/Users/user/root_dir/data/geojson/districts.geojson"

In [6]:
# all path
csv_path = "C:/Users/ASUS/Downloads/dist-report-v3.csv"
dist_lt_ln_path = "C:/Users/user/root_dir/data/nct-distr/dist-report-v3.csv"
zipcode_json_path = "C:/Users/ASUS/Downloads/ajax.json"
geo_distric_path = "data_source/districts.geojson" #from https://github.com/chingchai/OpenGISData-Thailand

In [4]:
os.getcwd()

'D:\\Onedrive\\SynMAR2020\\GitHub\\nct_dist\\nct_distr'

In [7]:
df = pd.read_csv(csv_path)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288522 entries, 0 to 288521
Data columns (total 25 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Unnamed: 0                                288522 non-null  int64  
 1   ReportDisburseNarcoticBeYear              288522 non-null  float64
 2   ReportDisburseNarcoticMonth               288522 non-null  float64
 3   ProductTypeName                           288522 non-null  object 
 4   MonthlyReportNarcoticName                 288522 non-null  object 
 5   MonthlyReportNarcoticBalanceForwardValue  288522 non-null  float64
 6   MonthlyReportNarcoticReceiveValue         288522 non-null  float64
 7   MonthlyReportNarcoticPaidValue            288522 non-null  float64
 8   MonthlyReportNarcoticRemainValue          288522 non-null  float64
 9   ReferCustomerName                         288522 non-null  object 
 10  CustomerAddressProvi

In [7]:
df['location_type'].value_counts()

location_type
ROOFTOP               146501
APPROXIMATE            69632
GEOMETRIC_CENTER       40027
RANGE_INTERPOLATED     32362
Name: count, dtype: int64

In [9]:
alldist_df.loc[alldist_df['MonthlyReportNarcoticName'].str.contains("alprazolam", case=False)]['MonthlyReportNarcoticName'].value_counts()

MonthlyReportNarcoticName
Alprazolam tablets 0.5 mg (Polipharm)                 12179
Alprazolam tablets 0.25 mg (Polipharm)                 8278
Alprazolam tablet 1.0 mg (Polipharm) (100 tab/box)     6996
Alprazolam tablets 1 mg (Pharmasant)                    570
Alprazolam tablets 0.5 mg (Pharmasant)                  156
Name: count, dtype: int64

In [8]:
alldist_alpra = df.loc[df['MonthlyReportNarcoticName'].str.contains("alprazolam", case=False)].copy()

In [12]:
alldist_alpra.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28179 entries, 6 to 288513
Data columns (total 25 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Unnamed: 0                                28179 non-null  int64  
 1   ReportDisburseNarcoticBeYear              28179 non-null  float64
 2   ReportDisburseNarcoticMonth               28179 non-null  float64
 3   ProductTypeName                           28179 non-null  object 
 4   MonthlyReportNarcoticName                 28179 non-null  object 
 5   MonthlyReportNarcoticBalanceForwardValue  28179 non-null  float64
 6   MonthlyReportNarcoticReceiveValue         28179 non-null  float64
 7   MonthlyReportNarcoticPaidValue            28179 non-null  float64
 8   MonthlyReportNarcoticRemainValue          28179 non-null  float64
 9   ReferCustomerName                         28179 non-null  object 
 10  CustomerAddressProvinceName           

In [7]:
overall_sum_alpra = alldist_alpra.groupby(by=['MonthlyReportNarcoticName'])['MonthlyReportNarcoticPaidValue'].sum().reset_index().sort_values(by=['MonthlyReportNarcoticPaidValue'],ascending=False)

In [11]:
overall_sum_alpra

,MonthlyReportNarcoticName,MonthlyReportNarcoticPaidValue
3,Alprazolam tablets 0.5 mg (Polipharm),15814979.0
0,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,8448290.0
1,Alprazolam tablets 0.25 mg (Polipharm),7918225.5
4,Alprazolam tablets 1 mg (Pharmasant),1241.0
2,Alprazolam tablets 0.5 mg (Pharmasant),359.0


In [13]:
alldist_alpra['fullAddress'].str.extract(r'(\d+)$').info()

<class 'pandas.core.frame.DataFrame'>
Index: 28179 entries, 6 to 288513
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       28179 non-null  object
dtypes: object(1)
memory usage: 440.3+ KB


In [9]:
# extract dist report address zipcode
alldist_alpra['zip_code'] = alldist_alpra['fullAddress'].str.extract(r'(\d+)$')

In [10]:
alldist_alpra.loc[alldist_alpra['zip_code'].isna()]

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code


In [17]:
alldist_alpra['zip_code'].value_counts().reset_index().sort_values(by=['zip_code'])

,zip_code,count
26,10100,284
1,10110,934
31,10120,265
42,10130,213
47,10140,184
...,...,...
88,95000,80
156,95110,36
127,95140,48
100,96000,60


In [24]:
alldist_alpra.loc[alldist_alpra['zip_code'] == '10350'].iloc[0].loc['fullAddress']

'124-126 ถนนรามคำแหง ซอยสมานมิตร แขวงสวนหลวง เขตสวนหลวง กรุงเทพมหานคร 10350'

In [26]:
df_ct_alprazip

,zip_code,count
26,10100,284
1,10110,934
31,10120,265
42,10130,213
47,10140,184
...,...,...
88,95000,80
156,95110,36
127,95140,48
100,96000,60


In [28]:
df_ct_alprazip.info()

<class 'pandas.core.frame.DataFrame'>
Index: 341 entries, 26 to 333
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   zip_code  341 non-null    int32
 1   count     341 non-null    int64
dtypes: int32(1), int64(1)
memory usage: 6.7 KB


<h1> skip this part as a legacy zipcode no longer used </h1>

In [22]:
zipcode_df = pd.read_csv(zipcode_path)

In [26]:
zipcode_df

,เขต,รหัสไปรษณีย์,หมายเหตุ
0,คลองเตย,10110,(ไปรษณีย์พระโขนง)\n ยกเว้น แขวงพระโขนง เฉพาะซอ...
1,คลองสาน,10600,(ไปรษณีย์สำเหร่)
2,คลองสามวา,10510,(ไปรษณีย์มีนบุรี)
3,คันนายาว,10230,(ไปรษณีย์จรเข้บัว)
4,จตุจักร,10900,NaN
5,จอมทอง,10150,(ไปรษณีย์บางขุนเทียน)
6,ดอนเมือง,10210,(ไปรษณีย์หลักสี่)
7,ดินแดง,10400,(ไปรษณีย์สามเสนใน)
8,ดุสิต,10300,ยกเว้น ภายในพระราชวังดุสิต สวนจิตรลดา ใช้รหัส ...
9,ตลิ่งชัน,10170,NaN


In [36]:
df_ct_alprazip.merge(
    zipcode_df,
    how = "left",
    left_on = ['zip_code'],
    right_on = ['รหัสไปรษณีย์']
)

,zip_code,count,เขต,รหัสไปรษณีย์,หมายเหตุ
0,10100,284,ป้อมปราบศัตรูพ่าย,10100.0,(ไปรษณีย์พลับพลาไชย)
1,10100,284,สัมพันธวงศ์,10100.0,(ไปรษณีย์พลับพลาไชย)
2,10110,934,คลองเตย,10110.0,(ไปรษณีย์พระโขนง)\n ยกเว้น แขวงพระโขนง เฉพาะซอ...
3,10110,934,วัฒนา,10110.0,(ไปรษณีย์พระโขนง)\n ยกเว้น แขวงพระโขนงเหนือ เฉ...
4,10120,265,บางคอแหลม,10120.0,(ไปรษณีย์ยานนาวา)
...,...,...,...,...,...
360,95000,80,NaN,NaN,NaN
361,95110,36,NaN,NaN,NaN
362,95140,48,NaN,NaN,NaN
363,96000,60,NaN,NaN,NaN


In [37]:
merged_df = df_ct_alprazip.merge(
    zipcode_df,
    how = "left",
    left_on = ['zip_code'],
    right_on = ['รหัสไปรษณีย์']
)

In [39]:
merged_df.loc[merged_df['เขต'].isna()]

,zip_code,count,เขต,รหัสไปรษณีย์,หมายเหตุ
7,10130,213,NaN,NaN,NaN
32,10270,268,NaN,NaN,NaN
33,10280,60,NaN,NaN,NaN
34,10290,24,NaN,NaN,NaN
39,10350,24,NaN,NaN,NaN
...,...,...,...,...,...
360,95000,80,NaN,NaN,NaN
361,95110,36,NaN,NaN,NaN
362,95140,48,NaN,NaN,NaN
363,96000,60,NaN,NaN,NaN


In [27]:
zipcode_df.loc[zipcode_df['รหัสไปรษณีย์'] == "10130"]

NameError: name 'zipcode_df' is not defined

<h1> The newer json from www.ไปรษณีย์รหัส.com is more reliable </h1>

In [14]:
zipcode_df2 = pd.read_json(zipcode_json_path)

In [15]:
zipcode_df2['zipcode'] = zipcode_df2['data'].str.extract(r'(\d+)$')

In [16]:
zipcode_df2.astype({'zipcode': 'int32'})

,data,zipcode
0,ต.กกกุง อ.เมืองสรวง จ.ร้อยเอ็ด 45220,45220
1,ต.กกดู่ อ.เมืองเลย จ.เลย 42000,42000
2,ต.กกตูม อ.ดงหลวง จ.มุกดาหาร 49140,49140
3,ต.กกทอง อ.เมืองเลย จ.เลย 42000,42000
4,ต.กกปลาซิว อ.ภูพาน จ.สกลนคร 47180,47180
...,...,...
7443,ต.ไสไทย อ.เมืองกระบี่ จ.กระบี่ 81000,81000
7444,ต.ไหล่ทุ่ง อ.ตระการพืชผล จ.อุบลราชธานี 34130,34130
7445,ต.ไหล่น่าน อ.เวียงสา จ.น่าน 55110,55110
7446,ต.ไหล่หิน อ.เกาะคา จ.ลำปาง 52130,52130


In [17]:
zipcode_df2['zipcode'] = zipcode_df2['zipcode'].astype('int32')

In [19]:
zipcode_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7448 entries, 0 to 7447
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   data     7448 non-null   object
 1   zipcode  7448 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 87.4+ KB


In [25]:
zipcode_df2.loc[zipcode_df2['data'].str.contains('สวนหลวง')]

,data,zipcode
4685,ต.สวนหลวง อ.อัมพวา จ.สมุทรสงคราม 75110,75110
4686,ต.สวนหลวง อ.กระทุ่มแบน จ.สมุทรสาคร 74110,74110
4687,ต.สวนหลวง อ.เขตสวนหลวง จ.กรุงเทพมหานคร 10250,10250
4688,ต.สวนหลวง อ.เฉลิมพระเกียรติ จ.นครศรีธรรมราช 80190,80190
4689,ต.สวนหลวง อ.เขตประเวศ จ.กรุงเทพมหานคร 10250,10250


In [26]:
zipcode_df2.loc[zipcode_df2['data'].str.contains('10350')]

,data,zipcode


In [35]:
zipcode_df2 = zipcode_df2.astype({'zipcode': 'int32'})

<h2>zip code correction in the main dataframe as the following</h2>

In [3]:
replace_zip = {
    "10350": "10250", # แขวงสวนหลวง เขตสวนหลวง กรุงเทพมหานคร
    "10570": "10540", # ตำบลบางเสาธง อำเภอบางเสาธง สมุทรปราการ
    "40002": "40000", # ตำบลในเมือง อำเภอเมืองขอนแก่น จังหวัดขอนแก่น
    "81210": "81000", # ตำบลอ่าวนาง อำเภอเมืองกระบี่ กระบี่
    "8300": "83000", # ตำบลวิชิต อำเภอเมืองภูเก็ต ภูเก็ต
    "9300": "93000" # ตำบลคูหาสวรรค์ อำเภอเมืองพัทลุง จังหวัดพัทลุง
}

### calculate affected records

In [13]:
alldist_alpra['zip_code'].isin(replace_zip.keys()).sum()

np.int64(152)

In [17]:
for i in replace_zip.keys():
    print(f'zip:{i}, {alldist_alpra["zip_code"].isin([i]).sum()}')

zip:10350, 24
zip:10570, 57
zip:40002, 48
zip:81210, 12
zip:8300, 7
zip:9300, 4


### replace existed values

In [23]:
alldist_alpra['zip_code'] = alldist_alpra['zip_code'].replace(replace_zip)

<h1>Overall summary merge</h1>

In [11]:
df_ct_alprazip = alldist_alpra['zip_code'].value_counts().reset_index().sort_values(by=['zip_code'])

In [12]:
df_ct_alprazip['zip_code'] = df_ct_alprazip['zip_code'].astype('int32')

In [24]:
merged_df = df_ct_alprazip.merge(
    zipcode_df2,
    how = "left",
    left_on = ['zip_code'],
    right_on = ['zipcode']
)

<h2>check NA</h2>

In [25]:
merged_df.loc[merged_df['data'].isna()]

,zip_code,count,data,zipcode
142,10350,24,NaN,NaN
191,10570,57,NaN,NaN
1480,40002,48,NaN,NaN
3043,81210,12,NaN,NaN
3061,8300,7,NaN,NaN
3243,9300,4,NaN,NaN


<h2>Extract district name from zipcode raw data</h2>

In [26]:
# \sอ.(.*)\sจ
zipcode_df2['district'] = zipcode_df2['data'].str.extract(r'\sอ.(.*)\sจ')

In [27]:
zipcode_df2['district'].isna().value_counts()

district
False    7448
Name: count, dtype: int64

In [28]:
clean_district_name = {
    "เมืองนครศรีธรรมร": "เมืองนครศรีธรรมราช",
    "เมืองสุราษฎร์ธาน": "เมืองสุราษฎร์ธานี",
    "เมืองประจวบคีรีข": "เมืองประจวบคีรีขันธ์",
    "ป่าพยอม": "ป่าพะยอม",
    "กรงปีนัง": "กรงปินัง",
    "ว่านใหญ่": "หว้านใหญ่"
}

In [29]:
reversed_district_name = {v: k for k, v in clean_district_name.items()}

In [43]:
reversed_district_name

{'เมืองนครศรีธรรมราช': 'เมืองนครศรีธรรมร',
 'เมืองสุราษฎร์ธานี': 'เมืองสุราษฎร์ธาน',
 'เมืองประจวบคีรีขันธ์': 'เมืองประจวบคีรีข',
 'ป่าพะยอม': 'ป่าพยอม',
 'กรงปินัง': 'กรงปีนัง',
 'หว้านใหญ่': 'ว่านใหญ่'}

In [30]:
# Prepare district 2 column for wrangling
zipcode_df2['district2'] = zipcode_df2['district'].str.replace(r'^เขต', '', regex=True)
zipcode_df2['district2'] = zipcode_df2['district2'].replace(reversed_district_name)

In [41]:
zipcode_df2

,data,zipcode,district,district2
0,ต.กกกุง อ.เมืองสรวง จ.ร้อยเอ็ด 45220,45220,เมืองสรวง,เมืองสรวง
1,ต.กกดู่ อ.เมืองเลย จ.เลย 42000,42000,เมืองเลย,เมืองเลย
2,ต.กกตูม อ.ดงหลวง จ.มุกดาหาร 49140,49140,ดงหลวง,ดงหลวง
3,ต.กกทอง อ.เมืองเลย จ.เลย 42000,42000,เมืองเลย,เมืองเลย
4,ต.กกปลาซิว อ.ภูพาน จ.สกลนคร 47180,47180,ภูพาน,ภูพาน
...,...,...,...,...
7443,ต.ไสไทย อ.เมืองกระบี่ จ.กระบี่ 81000,81000,เมืองกระบี่,เมืองกระบี่
7444,ต.ไหล่ทุ่ง อ.ตระการพืชผล จ.อุบลราชธานี 34130,34130,ตระการพืชผล,ตระการพืชผล
7445,ต.ไหล่น่าน อ.เวียงสา จ.น่าน 55110,55110,เวียงสา,เวียงสา
7446,ต.ไหล่หิน อ.เกาะคา จ.ลำปาง 52130,52130,เกาะคา,เกาะคา


In [31]:
gdf = geopandas.read_file(geo_distric_path)

In [32]:
merge3 = zipcode_df2.merge(
    gdf,
    how = 'left',
    left_on = 'district2',
    right_on = 'amp_th'
)

In [33]:
merge3 = merge3.drop_duplicates()

In [45]:
merge3

,data,zipcode,district,district2,amp_code,amp_th,amp_en,pro_code,pro_th,pro_en,reg_nesdb,reg_royin,perimeter,area_sqkm,geometry
0,ต.กกกุง อ.เมืองสรวง จ.ร้อยเอ็ด 45220,45220,เมืองสรวง,เมืองสรวง,4512,เมืองสรวง,Mueang Suang,45,ร้อยเอ็ด,Roi Et,Northeast,Northeast,58.122528,153.250769,"MULTIPOLYGON (((103.75082 15.73737, 103.7219 1..."
1,ต.กกดู่ อ.เมืองเลย จ.เลย 42000,42000,เมืองเลย,เมืองเลย,4201,เมืองเลย,Mueang Loei,42,เลย,Loei,Northeast,Northeast,180.264985,1271.529952,"MULTIPOLYGON (((101.76247 17.40091, 101.75396 ..."
2,ต.กกตูม อ.ดงหลวง จ.มุกดาหาร 49140,49140,ดงหลวง,ดงหลวง,4904,ดงหลวง,Dong Luang,49,มุกดาหาร,Mukdahan,Northeast,Northeast,218.006461,897.704792,"MULTIPOLYGON (((104.57583 16.75619, 104.55689 ..."
3,ต.กกทอง อ.เมืองเลย จ.เลย 42000,42000,เมืองเลย,เมืองเลย,4201,เมืองเลย,Mueang Loei,42,เลย,Loei,Northeast,Northeast,180.264985,1271.529952,"MULTIPOLYGON (((101.76247 17.40091, 101.75396 ..."
4,ต.กกปลาซิว อ.ภูพาน จ.สกลนคร 47180,47180,ภูพาน,ภูพาน,4718,ภูพาน,Phu Phan,47,สกลนคร,Sakon Nakhon,Northeast,Northeast,129.667665,708.302704,"MULTIPOLYGON (((104.05005 16.86694, 104.04235 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7541,ต.ไสไทย อ.เมืองกระบี่ จ.กระบี่ 81000,81000,เมืองกระบี่,เมืองกระบี่,8101,เมืองกระบี่,Mueang Krabi,81,กระบี่,Krabi,South,South,203.160867,906.857059,"MULTIPOLYGON (((98.86454 7.61216, 98.86241 7.6..."
7542,ต.ไหล่ทุ่ง อ.ตระการพืชผล จ.อุบลราชธานี 34130,34130,ตระการพืชผล,ตระการพืชผล,3411,ตระการพืชผล,Trakan Phuet Phon,34,อุบลราชธานี,Ubon Ratchathani,Northeast,Northeast,173.952730,1029.512650,"MULTIPOLYGON (((105.20037 15.54484, 105.2017 1..."
7543,ต.ไหล่น่าน อ.เวียงสา จ.น่าน 55110,55110,เวียงสา,เวียงสา,5507,เวียงสา,Wiang Sa,55,น่าน,Nan,UpperNorth,North,269.285521,2162.662327,"MULTIPOLYGON (((101.09153 18.49925, 101.09338 ..."
7544,ต.ไหล่หิน อ.เกาะคา จ.ลำปาง 52130,52130,เกาะคา,เกาะคา,5203,เกาะคา,Ko Kha,52,ลำปาง,Lampang,UpperNorth,North,114.341203,513.552125,"MULTIPOLYGON (((99.43166 18.12897, 99.43495 18..."


In [34]:
merge3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7546 entries, 0 to 7545
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   data       7546 non-null   object  
 1   zipcode    7546 non-null   int32   
 2   district   7546 non-null   object  
 3   district2  7546 non-null   object  
 4   amp_code   7546 non-null   object  
 5   amp_th     7546 non-null   object  
 6   amp_en     7546 non-null   object  
 7   pro_code   7546 non-null   object  
 8   pro_th     7546 non-null   object  
 9   pro_en     7546 non-null   object  
 10  reg_nesdb  7546 non-null   object  
 11  reg_royin  7546 non-null   object  
 12  perimeter  7546 non-null   float64 
 13  area_sqkm  7546 non-null   float64 
 14  geometry   7546 non-null   geometry
dtypes: float64(2), geometry(1), int32(1), object(11)
memory usage: 854.9+ KB


<h1>Wrangling join back</h1>

<h2>Approach 1: [External validity] merge with external zip code details</h2>

In [74]:
def iterzipcode(df_left, zipmerge):
    li = []
    multiple_case_index = []
    unknown_case_index = []
    for i, v in df_left.iterrows():
        full_address = v['fullAddress']
        zipcode = int(v['zip_code'])
        f_zip = zipmerge.loc[zipmerge['zipcode'] == zipcode]
        if len(f_zip) >1 :
            for sliced_index, row in f_zip.iterrows():
                district_name = row['district2']
                if district_name in full_address:
                    li.append(district_name)
                    continue
                else:
                    print(f"index: {i}, 1:m case no mach")
                    li.append(np.nan)
                    multiple_case_index.append(i)
                    continue
        elif len(f_zip) == 1 :
            li.append(f_zip.iloc[0]['district2'])
        else:
            print(f"index{i},An unknown case")
            li.append(np.nan)
            unknown_case_index.append(i)
    return {"district_name": li, "multiple_case_index": multiple_case_index, "unknown_case_index": unknown_case_index}

In [35]:
def iterzipcode2(df_left, zipmerge):
    df_copy = df_left.copy()
    multiple_case_index = []
    unknown_case_index = []
    for i, v in df_copy.iterrows():
        full_address = v['fullAddress']
        zipcode = int(v['zip_code'])
        f_zip = zipmerge.loc[zipmerge['zipcode'] == zipcode]
        if len(f_zip) >1 :
            for sliced_index, row in f_zip.iterrows():
                district_name = row['district2']
                if district_name in full_address:
                    df_copy.at[i,'district_name'] = district_name
                    break
            continue
        elif len(f_zip) == 1 :
            df_copy.at[i,'district_name'] = f_zip.iloc[0]['district2']
            continue
        else:
            pass
    return df_copy

In [36]:
alldist_alpra['district_name'] = np.nan
alldist_alpra['district_name'] = alldist_alpra['district_name'].astype(str)

<h3>Perform dimensional reduction to district level</h3>

In [37]:
sel_m3 = merge3[['zipcode','district2']].drop_duplicates()

In [97]:
wran_district2 = iterzipcode2(alldist_alpra, sel_m3)

<h3>reheck na merge record's address and zip codes</h3>

In [102]:
wran_district2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28179 entries, 6 to 288513
Data columns (total 27 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Unnamed: 0                                28179 non-null  int64  
 1   ReportDisburseNarcoticBeYear              28179 non-null  float64
 2   ReportDisburseNarcoticMonth               28179 non-null  float64
 3   ProductTypeName                           28179 non-null  object 
 4   MonthlyReportNarcoticName                 28179 non-null  object 
 5   MonthlyReportNarcoticBalanceForwardValue  28179 non-null  float64
 6   MonthlyReportNarcoticReceiveValue         28179 non-null  float64
 7   MonthlyReportNarcoticPaidValue            28179 non-null  float64
 8   MonthlyReportNarcoticRemainValue          28179 non-null  float64
 9   ReferCustomerName                         28179 non-null  object 
 10  CustomerAddressProvinceName           

In [103]:
wran_district2.loc[wran_district2['district_name'] == 'nan']

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name
5411,5444,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,250.0,0.0,250.0,0.0,ซันซายน์มายด์คลินิกเฉพาะทางด้านเวชกรรมจิตเวช,กรุงเทพมหานคร,คลินิก,RECORD,1/03/2024 15:55 น.,นางสาว จริยา บัวศรี,3/1/2024,cbc016ee-18df-4209-8fb3-5d6ff6f3b754,d891499a-ccea-4853-8e21-f3aed590eb7e,21 โครงการแพลทินัมเพลส ชั้น 1 ห้องเลขที่ 1A-8 ...,"1 ถ. วัชรพล Khwaeng Tha Raeng, Khet Bang Khen,...",ROOFTOP,ChIJv9nfyTR9HTERuS9v3wtAOWY,13.867306,100.646396,street_address,10230,nan
5416,5449,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),200.0,500.0,500.0,200.0,ซันซายน์มายด์คลินิกเฉพาะทางด้านเวชกรรมจิตเวช,กรุงเทพมหานคร,คลินิก,RECORD,1/03/2024 15:55 น.,นางสาว จริยา บัวศรี,3/1/2024,e76a8d4b-54bc-44b5-84e1-5e90a6ec3d5c,d891499a-ccea-4853-8e21-f3aed590eb7e,21 โครงการแพลทินัมเพลส ชั้น 1 ห้องเลขที่ 1A-8 ...,"1 ถ. วัชรพล Khwaeng Tha Raeng, Khet Bang Khen,...",ROOFTOP,ChIJv9nfyTR9HTERuS9v3wtAOWY,13.867306,100.646396,street_address,10230,nan
7196,7235,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),100.0,0.0,0.0,100.0,บลอสชั้นมคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/02/2024 14:02 น.,นางสาว จริยา บัวศรี,2/27/2024,4b122115-119e-476a-9696-b18982af0288,4b122115-119e-476a-9696-b18982af0288,88 โครงการ Stadium one ชั้น 1 ห้องเลขที่ E107...,"56 ซอย จุฬาลงกรณ์ 5 Khwaeng Wang Mai, Pathum W...",RANGE_INTERPOLATED,EnI1NiDguIvguK3guKIg4LiI4Li44Lis4Liy4Lil4LiH4L...,13.742439,100.524138,street_address,10110,nan
7197,7236,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),100.0,0.0,0.0,100.0,บลอสชั้นมคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/02/2024 14:02 น.,นางสาว จริยา บัวศรี,2/27/2024,4b122115-119e-476a-9696-b18982af0288,d59ce442-4261-4698-8c5d-627b2f362e86,88 โครงการ Stadium one ชั้น 1 ห้องเลขที่ E107...,"56 ซอย จุฬาลงกรณ์ 5 Khwaeng Wang Mai, Pathum W...",RANGE_INTERPOLATED,EnI1NiDguIvguK3guKIg4LiI4Li44Lis4Liy4Lil4LiH4L...,13.742439,100.524138,street_address,10110,nan
7198,7237,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,100.0,0.0,0.0,100.0,บลอสชั้นมคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/02/2024 14:02 น.,นางสาว จริยา บัวศรี,2/27/2024,a8b23683-7ad0-49b4-bb60-f7cfd6fa4b33,4b122115-119e-476a-9696-b18982af0288,88 โครงการ Stadium one ชั้น 1 ห้องเลขที่ E107...,"56 ซอย จุฬาลงกรณ์ 5 Khwaeng Wang Mai, Pathum W...",RANGE_INTERPOLATED,EnI1NiDguIvguK3guKIg4LiI4Li44Lis4Liy4Lil4LiH4L...,13.742439,100.524138,street_address,10110,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284066,285041,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,0.0,0.0,0.0,0.0,เดอะมายด์บางกอก คลินิกเฉพาะทางด้านเวชกรรมจิตเว...,กรุงเทพมหานคร,คลินิก,RECORD,13/02/2025 9:59 น.,นาง จริยา บัวศรี,2/13/2025,045421fb-9c39-47cf-9df8-079b759c392b,a3219a38-4f44-44b0-b72d-e62a593e48b6,320 อาคารตั้งฮั่วปัก ชั้น3 ถนนพระราม 4 แขวงมหา...,"Thanon Rama IV, Krung Thep Maha Nakhon, Thailand",GEOMETRIC_CENTER,ChIJDQJPcxif4jARSFIFFIkXftQ,13.723435,100.551410,route,10400,nan
284068,285043,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),0.0,0.0,0.0,0.0,เดอะมายด์บางกอก คลินิกเฉพาะทางด้านเวชกรรมจิตเว...,กรุงเทพมหานคร,คลินิก,RECORD,13/02/2025 9:59 น.,นาง จริยา บัวศรี,2/13/2025,671362e3-c90c-4b45-84e5-c445b87aa822,a3219a38-4f44-44b0-b72d-e62a593e48b6,320 อาคารตั้งฮั่วปัก ชั้น3 ถนนพระราม 4 แข

In [98]:
wran_district2.loc[wran_district2['district_name'].isna()][['fullAddress','zip_code']].drop_duplicates()

,fullAddress,zip_code


<h3> Correct district name and zipcode using internal validity and external validity</h3>

In [40]:
def corr_ampcode(corr_df, mergezip_df):
    corrdf = corr_df.copy()
    for i, v in corrdf.iterrows():
        ampcode = str(v['amp_code'])
        print(f"searching for ampcode {ampcode}")
        d_n = mergezip_df.loc[mergezip_df['amp_code'] == ampcode].iloc[0]['district2']
        corrdf.at[i,'district_name'] = d_n
    return corrdf

In [43]:
corr_per_index = {
    5411: {
        "zipcode": 10220,
        "amp_code": 1005
    }, # 21 ถ. วัชรพล Tha Raeng, Bang Khen, Bangkok 10220
    7196: {
        "zipcode": 10330,
        "amp_code": 1007
    },  # 719,721,723,725,727 Chulalongkorn 4 Alley, Wang Mai, Pathum Wan, Bangkok 10330
    7534: {
        "zipcode": 10240,
        "amp_code": 1027
    }, # 9 Rachada-Ramintra Rd, Khwaeng Nuanchan, Khet Bueng Kum, Krung Thep Maha Nakhon 10230
    14271: {
        "zipcode": 10240,
        "amp_code": 1027
    }, # 44/505 Thanon Nawamin, Khwaeng Nuanchan, Khet Bueng Kum, Krung Thep Maha Nakhon 10230, Thailand
    15482: {
        "zipcode": 10500,
        "amp_code": 1004
    }, # 320 Rama IV Rd, Maha Phruttharam, Bang Rak, Bangkok 10500
    20806: {
        "zipcode": 10110,
        "amp_code": 1033
    }, # 2024 ถ. ทางรถไฟสายเก่าปากน้ำ Khwaeng Phra Khanong, Khet Khlong Toei, Krung Thep Maha Nakhon 10110, Thailand
    21094: {
        "zipcode": 12000,
        "amp_code": 1301
    }, # 10/88 หมู่4 ถนนกรุงเทพฯ Bang Khayaeng, อำเภอเมือง, Pathum Thani 12000
    22053: {
        "zipcode": 10260,
        "amp_code": 1009
    } # 86/5 Sukhumvit 89 Alley, Bang Chak, Phra Khanong, Bangkok 10260
}

In [45]:
corr_per_index_df = pd.DataFrame(corr_per_index).transpose(copy=True)

In [46]:
corr_amp_df = corr_ampcode(corr_per_index_df, merge3)

searching for ampcode 1005
searching for ampcode 1007
searching for ampcode 1027
searching for ampcode 1027
searching for ampcode 1004
searching for ampcode 1033
searching for ampcode 1301
searching for ampcode 1009


In [99]:
corr_amp_df

,zipcode,amp_code,district_name
5411,10220,1005,บางเขน
7196,10330,1007,ปทุมวัน
7534,10240,1027,บึงกุ่ม
14271,10240,1027,บึงกุ่ม
15482,10500,1004,บางรัก
20806,10110,1033,คลองเตย
21094,12000,1301,เมืองปทุมธานี
22053,10260,1009,พระโขนง


In [104]:
# correct 'nan' back to float values
wran_district2['district_name'] = wran_district2['district_name'].replace(to_replace='nan',value=np.nan)

In [105]:
wran_district2.loc[wran_district2['district_name'] == 'nan']

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name


In [106]:
wran_district2.loc[wran_district2['district_name'].isna()]

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name
5411,5444,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,250.0,0.0,250.0,0.0,ซันซายน์มายด์คลินิกเฉพาะทางด้านเวชกรรมจิตเวช,กรุงเทพมหานคร,คลินิก,RECORD,1/03/2024 15:55 น.,นางสาว จริยา บัวศรี,3/1/2024,cbc016ee-18df-4209-8fb3-5d6ff6f3b754,d891499a-ccea-4853-8e21-f3aed590eb7e,21 โครงการแพลทินัมเพลส ชั้น 1 ห้องเลขที่ 1A-8 ...,"1 ถ. วัชรพล Khwaeng Tha Raeng, Khet Bang Khen,...",ROOFTOP,ChIJv9nfyTR9HTERuS9v3wtAOWY,13.867306,100.646396,street_address,10230,NaN
5416,5449,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),200.0,500.0,500.0,200.0,ซันซายน์มายด์คลินิกเฉพาะทางด้านเวชกรรมจิตเวช,กรุงเทพมหานคร,คลินิก,RECORD,1/03/2024 15:55 น.,นางสาว จริยา บัวศรี,3/1/2024,e76a8d4b-54bc-44b5-84e1-5e90a6ec3d5c,d891499a-ccea-4853-8e21-f3aed590eb7e,21 โครงการแพลทินัมเพลส ชั้น 1 ห้องเลขที่ 1A-8 ...,"1 ถ. วัชรพล Khwaeng Tha Raeng, Khet Bang Khen,...",ROOFTOP,ChIJv9nfyTR9HTERuS9v3wtAOWY,13.867306,100.646396,street_address,10230,NaN
7196,7235,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),100.0,0.0,0.0,100.0,บลอสชั้นมคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/02/2024 14:02 น.,นางสาว จริยา บัวศรี,2/27/2024,4b122115-119e-476a-9696-b18982af0288,4b122115-119e-476a-9696-b18982af0288,88 โครงการ Stadium one ชั้น 1 ห้องเลขที่ E107...,"56 ซอย จุฬาลงกรณ์ 5 Khwaeng Wang Mai, Pathum W...",RANGE_INTERPOLATED,EnI1NiDguIvguK3guKIg4LiI4Li44Lis4Liy4Lil4LiH4L...,13.742439,100.524138,street_address,10110,NaN
7197,7236,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),100.0,0.0,0.0,100.0,บลอสชั้นมคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/02/2024 14:02 น.,นางสาว จริยา บัวศรี,2/27/2024,4b122115-119e-476a-9696-b18982af0288,d59ce442-4261-4698-8c5d-627b2f362e86,88 โครงการ Stadium one ชั้น 1 ห้องเลขที่ E107...,"56 ซอย จุฬาลงกรณ์ 5 Khwaeng Wang Mai, Pathum W...",RANGE_INTERPOLATED,EnI1NiDguIvguK3guKIg4LiI4Li44Lis4Liy4Lil4LiH4L...,13.742439,100.524138,street_address,10110,NaN
7198,7237,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,100.0,0.0,0.0,100.0,บลอสชั้นมคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/02/2024 14:02 น.,นางสาว จริยา บัวศรี,2/27/2024,a8b23683-7ad0-49b4-bb60-f7cfd6fa4b33,4b122115-119e-476a-9696-b18982af0288,88 โครงการ Stadium one ชั้น 1 ห้องเลขที่ E107...,"56 ซอย จุฬาลงกรณ์ 5 Khwaeng Wang Mai, Pathum W...",RANGE_INTERPOLATED,EnI1NiDguIvguK3guKIg4LiI4Li44Lis4Liy4Lil4LiH4L...,13.742439,100.524138,street_address,10110,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284066,285041,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,0.0,0.0,0.0,0.0,เดอะมายด์บางกอก คลินิกเฉพาะทางด้านเวชกรรมจิตเว...,กรุงเทพมหานคร,คลินิก,RECORD,13/02/2025 9:59 น.,นาง จริยา บัวศรี,2/13/2025,045421fb-9c39-47cf-9df8-079b759c392b,a3219a38-4f44-44b0-b72d-e62a593e48b6,320 อาคารตั้งฮั่วปัก ชั้น3 ถนนพระราม 4 แขวงมหา...,"Thanon Rama IV, Krung Thep Maha Nakhon, Thailand",GEOMETRIC_CENTER,ChIJDQJPcxif4jARSFIFFIkXftQ,13.723435,100.551410,route,10400,NaN
284068,285043,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),0.0,0.0,0.0,0.0,เดอะมายด์บางกอก คลินิกเฉพาะทางด้านเวชกรรมจิตเว...,กรุงเทพมหานคร,คลินิก,RECORD,13/02/2025 9:59 น.,นาง จริยา บัวศรี,2/13/2025,671362e3-c90c-4b45-84e5-c445b87aa822,a3219a38-4f44-44b0-b72d-e62a593e48b6,320 อาคารตั้งฮั่วปัก ชั้น3 ถนนพระราม 4 แข

In [107]:
concat_district_col = wran_district2['district_name'].combine_first(corr_amp_df['district_name']).ffill()

In [110]:
concat_district_col.reset_index().loc[concat_district_col.reset_index()['district_name'].isna()]

,index,district_name


In [111]:
wran_district2['district_name'] = concat_district_col

In [96]:
wran_district2.loc[wran_district2['district_name'].isna()]

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name


In [160]:
wran_district2

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,...,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name
6,6,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,...,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์
31,31,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,...,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้
54,54,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,...,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง
56,56,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,...,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง
61,61,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,...,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288486,289469,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,...,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส
288487,289470,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,...,d5ca07fe-e5b9-46b9-aeae-e2b3a69552d7,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส
288499,289482,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),6391.0,0.0,2386.0,4005.0,โรงพยาบาล พญาไท 2 โรงพยาบาลทั่วไปขนาดใหญ่,...,a052a759-7cfa-4daf-b965-d05043a5acc5,943-943/1 ถนนพหลโยธิน แขวงพญาไท เขตพญาไท กรุงเ...,"943 Phahon Yothin Road, Phaya Thai, Khet Phaya...",ROOFTOP,ChIJDeNU9K-e4jARO9-jMvcxZlA,13.769745,100.540678,"premise,street_address",10400,พญาไท
288503,289486,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,9345.0,0.0,3748.0,5597.0,โรงพยาบาล พญาไท 2 โรงพยาบาลทั่วไปขนาดใหญ่,...,a052a759-7cfa-4daf-b965-d05043a5acc5,943-943/1 ถนนพหลโยธิน แขวงพญาไท เขตพญาไท กรุงเ...,"943 Phahon Yothin Road, Phaya Thai, Khet Phaya...",ROOFTOP,ChIJDeNU9K-e4jARO9-jMvcxZlA,13.769745,100.540678,"premise,street_address",10400,พญาไท


In [51]:
wran_district2.loc[wran_district2['district_name'].isna()][['zip_code',	'district_name']].drop_duplicates()

,zip_code,district_name


In [52]:
corr_per_index_df['district_name'] = np.nan

In [53]:
corr_per_index_df['amp_code'] = corr_per_index_df['amp_code'].astype('str')

In [54]:
corr_per_index_df

,zipcode,amp_code,district_name
5411,10220,1005,NaN
7196,10330,1007,NaN
7534,10240,1027,NaN
14271,10240,1027,NaN
15482,10500,1004,NaN
20806,10110,1033,NaN
21094,12000,1301,NaN
22053,10260,1009,NaN


<h3>The following cells are used to check for correction, debugging purposes and examples.</h3>

In [55]:
corr_per_index_df.iloc[0]

zipcode          10220
amp_code          1005
district_name      NaN
Name: 5411, dtype: object

In [99]:
corr_per_index[22053]['amp_code']

1009

In [91]:
wran_district2.loc[22053,'formatted_address']

'2 Soi Sukhumvit 89, Khwaeng Bang Chak, Khet Phra Khanong, Krung Thep Maha Nakhon 10260, Thailand'

In [92]:
wran_district2.loc[22053,'fullAddress']

'86/5 ชั้น2-3 ซอยสุขุมวิท 89 แขวงบางจาก เขตพระโขนง กรุงเทพมหานคร 10110'

In [56]:
sel_m3.loc[sel_m3['zipcode'] == 10260]

,zipcode,district2
2446,10260,พระโขนง
2484,10260,บางนา


In [89]:
sel_m3.loc[sel_m3['district2'].str.contains('เมืองปทุมธานี')]

,zipcode,district2
2399,12000,เมืองปทุมธานี


In [52]:
sel_m3.loc[sel_m3['zipcode'] == 10100].iloc[0]['district2'] in wran_district2.loc[wran_district2['district_name'].isna()].iloc[0]['fullAddress']

True

In [49]:
wran_district2.loc[wran_district2['district_name'].isna()].iloc[0]

Unnamed: 0                                                                               5444
ReportDisburseNarcoticBeYear                                                           2567.0
ReportDisburseNarcoticMonth                                                               1.0
ProductTypeName                                               (วจ. 2) วัตถุออกฤทธิ์ในประเภท 2
MonthlyReportNarcoticName                   Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...
MonthlyReportNarcoticBalanceForwardValue                                                250.0
MonthlyReportNarcoticReceiveValue                                                         0.0
MonthlyReportNarcoticPaidValue                                                          250.0
MonthlyReportNarcoticRemainValue                                                          0.0
ReferCustomerName                                ซันซายน์มายด์คลินิกเฉพาะทางด้านเวชกรรมจิตเวช
CustomerAddressProvinceName                                 

In [51]:
wran_district2.loc[wran_district2['district_name'].isna()].iloc[0]['fullAddress']

'21 โครงการแพลทินัมเพลส ชั้น 1 ห้องเลขที่ 1A-8 ถนนวัชรพล แขวงท่าแร้ง เขตบางเขน กรุงเทพมหานคร 10230'

In [ ]:
# 21 ถ. วัชรพล Tha Raeng, Bang Khen, Bangkok 10220

In [95]:
merge3.loc[merge3['zipcode'] == 10260]

,data,zipcode,district,district2,amp_code,amp_th,amp_en,pro_code,pro_th,pro_en,reg_nesdb,reg_royin,perimeter,area_sqkm,geometry
2446,ต.บางจาก อ.เขตพระโขนง จ.กรุงเทพมหานคร 10260,10260,เขตพระโขนง,พระโขนง,1009,พระโขนง,Phra Khanong,10,กรุงเทพมหานคร,Bangkok,Central,Central,16.546180,13.685730,"MULTIPOLYGON (((100.63223 13.70472, 100.63411 ..."
2484,ต.บางนา อ.เขตบางนา จ.กรุงเทพมหานคร 10260,10260,เขตบางนา,บางนา,1047,บางนา,Bang Na,10,กรุงเทพมหานคร,Bangkok,Central,Central,19.736196,16.459202,"MULTIPOLYGON (((100.62436 13.6845, 100.62594 1..."


In [120]:
merge3.loc[merge3['amp_code'] == "1009"]

,data,zipcode,district,district2,amp_code,amp_th,amp_en,pro_code,pro_th,pro_en,reg_nesdb,reg_royin,perimeter,area_sqkm,geometry
2446,ต.บางจาก อ.เขตพระโขนง จ.กรุงเทพมหานคร 10260,10260,เขตพระโขนง,พระโขนง,1009,พระโขนง,Phra Khanong,10,กรุงเทพมหานคร,Bangkok,Central,Central,16.54618,13.68573,"MULTIPOLYGON (((100.63223 13.70472, 100.63411 ..."


In [44]:
wran_district2.loc[wran_district2['district_name'].isna()]

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name
6,6,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,3/16/2024,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,NaN
31,31,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,3/14/2024,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,NaN
54,54,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2/28/2024,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,NaN
56,56,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,NaN
61,61,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288486,289469,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,NaN
288487,289470,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,d5ca07fe-e5b9-46b9-aeae-e2b3a69552d7,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.8238

In [27]:
merge3

,data,zipcode,district,district2,amp_code,amp_th,amp_en,pro_code,pro_th,pro_en,reg_nesdb,reg_royin,perimeter,area_sqkm,geometry
0,ต.กกกุง อ.เมืองสรวง จ.ร้อยเอ็ด 45220,45220,เมืองสรวง,เมืองสรวง,4512,เมืองสรวง,Mueang Suang,45,ร้อยเอ็ด,Roi Et,Northeast,Northeast,58.122528,153.250769,"MULTIPOLYGON (((103.75082 15.73737, 103.7219 1..."
1,ต.กกดู่ อ.เมืองเลย จ.เลย 42000,42000,เมืองเลย,เมืองเลย,4201,เมืองเลย,Mueang Loei,42,เลย,Loei,Northeast,Northeast,180.264985,1271.529952,"MULTIPOLYGON (((101.76247 17.40091, 101.75396 ..."
2,ต.กกตูม อ.ดงหลวง จ.มุกดาหาร 49140,49140,ดงหลวง,ดงหลวง,4904,ดงหลวง,Dong Luang,49,มุกดาหาร,Mukdahan,Northeast,Northeast,218.006461,897.704792,"MULTIPOLYGON (((104.57583 16.75619, 104.55689 ..."
3,ต.กกทอง อ.เมืองเลย จ.เลย 42000,42000,เมืองเลย,เมืองเลย,4201,เมืองเลย,Mueang Loei,42,เลย,Loei,Northeast,Northeast,180.264985,1271.529952,"MULTIPOLYGON (((101.76247 17.40091, 101.75396 ..."
4,ต.กกปลาซิว อ.ภูพาน จ.สกลนคร 47180,47180,ภูพาน,ภูพาน,4718,ภูพาน,Phu Phan,47,สกลนคร,Sakon Nakhon,Northeast,Northeast,129.667665,708.302704,"MULTIPOLYGON (((104.05005 16.86694, 104.04235 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7541,ต.ไสไทย อ.เมืองกระบี่ จ.กระบี่ 81000,81000,เมืองกระบี่,เมืองกระบี่,8101,เมืองกระบี่,Mueang Krabi,81,กระบี่,Krabi,South,South,203.160867,906.857059,"MULTIPOLYGON (((98.86454 7.61216, 98.86241 7.6..."
7542,ต.ไหล่ทุ่ง อ.ตระการพืชผล จ.อุบลราชธานี 34130,34130,ตระการพืชผล,ตระการพืชผล,3411,ตระการพืชผล,Trakan Phuet Phon,34,อุบลราชธานี,Ubon Ratchathani,Northeast,Northeast,173.952730,1029.512650,"MULTIPOLYGON (((105.20037 15.54484, 105.2017 1..."
7543,ต.ไหล่น่าน อ.เวียงสา จ.น่าน 55110,55110,เวียงสา,เวียงสา,5507,เวียงสา,Wiang Sa,55,น่าน,Nan,UpperNorth,North,269.285521,2162.662327,"MULTIPOLYGON (((101.09153 18.49925, 101.09338 ..."
7544,ต.ไหล่หิน อ.เกาะคา จ.ลำปาง 52130,52130,เกาะคา,เกาะคา,5203,เกาะคา,Ko Kha,52,ลำปาง,Lampang,UpperNorth,North,114.341203,513.552125,"MULTIPOLYGON (((99.43166 18.12897, 99.43495 18..."


In [32]:
sel_m3.drop_duplicates()['zipcode'].value_counts()

zipcode
60150    4
94190    4
45000    4
31110    4
71220    3
        ..
30110    1
13140    1
35150    1
72160    1
25140    1
Name: count, Length: 955, dtype: int64

In [ ]:
sel_m3.drop_duplicates()

In [36]:
sel_m3.loc[sel_m3['zipcode'] == 60150]

,zipcode,district2
797,60150,ชุมตาบง
3287,60150,ลาดยาว
4283,60150,แม่วงก์
6880,60150,แม่เปิน


In [57]:
merge3['zipcode'].value_counts().reset_index()

,zipcode,count
0,31110,40
1,45000,33
2,30230,33
3,18000,31
4,31000,31
...,...,...
950,25240,1
951,84330,1
952,52230,1
953,95170,1


In [69]:
merge3.loc[merge3['zipcode'] ==31110]

,data,zipcode,district,district2,amp_code,amp_th,amp_en,pro_code,pro_th,pro_en,reg_nesdb,reg_royin,perimeter,area_sqkm,geometry
240,ต.ก้านเหลือง อ.นางรอง จ.บุรีรัมย์ 31110,31110,นางรอง,นางรอง,3104,นางรอง,Nang Rong,31,บุรีรัมย์,Buriram,Northeast,Northeast,163.127587,758.714013,"MULTIPOLYGON (((102.73824 14.49561, 102.71005 ..."
784,ต.ชำนิ อ.ชำนิ จ.บุรีรัมย์ 31110,31110,ชำนิ,ชำนิ,3118,ชำนิ,Chamni,31,บุรีรัมย์,Buriram,Northeast,Northeast,84.905673,240.178748,"MULTIPOLYGON (((102.90582 14.72488, 102.90533 ..."
812,ต.ชุมแสง อ.นางรอง จ.บุรีรัมย์ 31110,31110,นางรอง,นางรอง,3104,นางรอง,Nang Rong,31,บุรีรัมย์,Buriram,Northeast,Northeast,163.127587,758.714013,"MULTIPOLYGON (((102.73824 14.49561, 102.71005 ..."
831,ต.ช่อผกา อ.ชำนิ จ.บุรีรัมย์ 31110,31110,ชำนิ,ชำนิ,3118,ชำนิ,Chamni,31,บุรีรัมย์,Buriram,Northeast,Northeast,84.905673,240.178748,"MULTIPOLYGON (((102.90582 14.72488, 102.90533 ..."
936,ต.ดงอีจาน อ.โนนสุวรรณ จ.บุรีรัมย์ 31110,31110,โนนสุวรรณ,โนนสุวรรณ,3117,โนนสุวรรณ,Non Suwan,31,บุรีรัมย์,Buriram,Northeast,Northeast,62.984802,200.432560,"MULTIPOLYGON (((102.62066 14.58229, 102.63309 ..."
1290,ต.ตาเป๊ก อ.เฉลิมพระเกียรติ จ.บุรีรัมย์ 31110,31110,เฉลิมพระเกียรติ,เฉลิมพระเกียรติ,5515,เฉลิมพระเกียรติ,Chaloem Phra Kiat,55,น่าน,Nan,UpperNorth,North,136.841321,565.847522,"MULTIPOLYGON (((101.21681 19.60242, 101.22556 ..."
1291,ต.ตาเป๊ก อ.เฉลิมพระเกียรติ จ.บุรีรัมย์ 31110,31110,เฉลิมพระเกียรติ,เฉลิมพระเกียรติ,3032,เฉลิมพระเกียรติ,Chaloem Phra Kiat,30,นครราชสีมา,Nakhon Ratchasima,Northeast,Northeast,96.284817,279.921469,"MULTIPOLYGON (((102.33127 14.93848, 102.33646 ..."
1292,ต.ตาเป๊ก อ.เฉลิมพระเกียรติ จ.บุรีรัมย์ 31110,31110,เฉลิมพระเกียรติ,เฉลิมพระเกียรติ,8023,เฉลิมพระเกียรติ,Chaloem Phra Kiat,80,นครศรีธรรมราช,Nakhon Si Thammarat,South,South,87.178310,180.363553,"MULTIPOLYGON (((100.08159 8.1468, 100.07885 8...."
1293,ต.ตาเป๊ก อ.เฉลิมพระเกียรติ จ.บุรีรัมย์ 31110,31110,เฉลิมพระเกียรติ,เฉลิมพระเกียรติ,1913,เฉลิมพระเกียรติ,Chaloem Phra Kiat,19,สระบุรี,Saraburi,Central,Central,81.109286,176.139358,"MULTIPOLYGON (((100.92822 14.57261, 100.9237 1..."
1294,ต.ตาเป๊ก อ.เฉลิมพระเกียรติ จ.บุรีรัมย์ 31110,31110,เฉลิมพระเกียรติ,เฉลิมพระเกียรติ,3123,เฉลิมพระเกียรติ,Chaloem Phra Kiat,31,บุรีรัมย์,Buriram,Northeast,Northeast,86.225959,235.106463,"MULTIPOLYGON (((102.94117 14.51393, 102.94703 ..."


<h2> Approach 2: [Internal validiy] verification</h2>

<h1>Compute aggregate for plotting</h1>

In [58]:
df_ct_alprazip['zip_code'] = df_ct_alprazip['zip_code'].astype({'zip_code': 'int32'}, errors='raise')

In [59]:
wran_district2

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name
6,6,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,3/16/2024,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์
31,31,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,3/14/2024,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้
54,54,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2/28/2024,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง
56,56,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง
61,61,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288486,289469,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส
288487,289470,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,d5ca07fe-e5b9-46b9-aeae-e2b3a69552d7,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQT

In [61]:
agg1_ct = wran_district2['district_name'].value_counts().reset_index()

In [62]:
agg1_ct.sum()

district_name    เมืองเชียงใหม่เมืองชลบุรีเมืองนนทบุรีพญาไทคลอง...
count                                                        28179
dtype: object

In [166]:
wran_district2['MonthlyReportNarcoticName'].value_counts().sum()

28179

In [63]:
wran_district2['SendReportDate'] = pd.to_datetime(wran_district2['SendReportDate'])

In [64]:
wran_district2['SendReportDate'].min()

Timestamp('2024-01-29 00:00:00')

In [173]:
wran_district2['SendReportDate'].min()

Timestamp('2024-01-29 00:00:00')

In [174]:
wran_district2['SendReportDate'].max()

Timestamp('2025-06-05 00:00:00')

In [178]:
agg1_ct.sort_values(by=['count'], ascending=False)

,district_name,count
0,เมืองเชียงใหม่,805
1,เมืองชลบุรี,598
2,เมืองนนทบุรี,590
3,พญาไท,539
4,คลองเตย,508
...,...,...
331,ลำปลายมาศ,9
332,สุวรรณคูหา,6
333,หนองแค,6
334,นากลาง,5


In [183]:
agg1_ct.loc[agg1_ct['district_name'].str.contains('จอม')]

,district_name,count
66,จอมทอง,131
236,จอมบึง,15


<h1>Plotting chloropleth map</h1>

In [68]:
def ampcode_lookup(main_df,ref_gdf):
    
    mdf = main_df.copy()
    mdf['amp_code'] = np.nan
    mdf['amp_code'] = mdf['amp_code'].astype(str)
    for i, v in mdf.iterrows():
        # 1. filter province name:
        pv_name = v['CustomerAddressProvinceName']
        dn = v['district_name']
        loc_pv = ref_gdf.loc[ref_gdf['pro_th'] == pv_name]
        # 2. filter district name:
        loc_dn = loc_pv.loc[loc_pv['amp_th'] == dn]
        if len(loc_dn) >0:
            mdf.at[i,'amp_code'] = loc_dn.iloc[0]['amp_code']

    return mdf

In [65]:
# load geojson data
with open(geo_distric_path, 'r', encoding='utf-8-sig') as f:
    geojson_data = json.load(f)

<h2>Feature to amp_th</h2>

In [77]:
# this feature district name which has drawbacks for non-unique names
fig = px.choropleth_map(
    agg1_ct,
    geojson = geojson_data,
    featureidkey="properties.amp_th",
    locations = 'district_name',
    color = 'count'
)
fig.show()

In [185]:
gdf.loc[gdf['amp_th'].str.contains('จอม')]

,amp_code,amp_th,amp_en,pro_code,pro_th,pro_en,reg_nesdb,reg_royin,perimeter,area_sqkm,geometry
20,3204,จอมพระ,Chom Phra,32,สุรินทร์,Surin,Northeast,Northeast,93.634225,331.486587,"MULTIPOLYGON (((103.72978 15.10946, 103.7385 1..."
37,7002,จอมบึง,Chom Bueng,70,ราชบุรี,Ratchaburi,West,West,167.065315,795.155584,"MULTIPOLYGON (((99.55624 13.70512, 99.56244 13..."
267,5002,จอมทอง,Chom Thong,50,เชียงใหม่,Chiang Mai,UpperNorth,North,183.442653,1108.029403,"MULTIPOLYGON (((98.69671 18.41125, 98.70951 18..."
440,1035,จอมทอง,Chom Thong,10,กรุงเทพมหานคร,Bangkok,Central,Central,22.406193,23.495326,"MULTIPOLYGON (((100.48795 13.68531, 100.4885 1..."


In [188]:
gdf.columns

Index(['amp_code', 'amp_th', 'amp_en', 'pro_code', 'pro_th', 'pro_en',
       'reg_nesdb', 'reg_royin', 'perimeter', 'area_sqkm', 'geometry'],
      dtype='object')

In [112]:
wran_district3 = ampcode_lookup(wran_district2, gdf)

In [197]:
wran_district2

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name
6,6,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,2024-03-16,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์
31,31,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,2024-03-14,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้
54,54,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2024-02-28,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง
56,56,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง
61,61,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288486,289469,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,2025-01-10,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส
288487,289470,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,2025-01-10,2ae8194a-7022-4837-88b3-9baaab86a58e,d5ca07fe-e5b9-46b9-aeae-e2b3a69552d7,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,C

In [113]:
wran_district3.loc[wran_district3['amp_code'].notna()]

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code
6,6,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,3/16/2024,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์,7205
31,31,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,3/14/2024,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้,8302
54,54,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2/28/2024,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง,7005
56,56,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
61,61,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288486,289469,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส,9601
288487,289470,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,d5ca07fe-e5b9-46b9-aeae-e2b3a69552d7,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha

In [70]:
wran_district3.loc[wran_district3['amp_code'].isna()][['zip_code',	'district_name']].drop_duplicates()

,zip_code,district_name


In [114]:
wran_district3.loc[wran_district3['amp_code'].isna()]

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code


<h3>The following cells are used to rechecking</h3>

In [208]:
wran_district3.loc[24796]

Unnamed: 0                                                                              24883
ReportDisburseNarcoticBeYear                                                           2567.0
ReportDisburseNarcoticMonth                                                               2.0
ProductTypeName                                               (วจ. 2) วัตถุออกฤทธิ์ในประเภท 2
MonthlyReportNarcoticName                              Alprazolam tablets 0.25 mg (Polipharm)
MonthlyReportNarcoticBalanceForwardValue                                               3525.0
MonthlyReportNarcoticReceiveValue                                                         0.0
MonthlyReportNarcoticPaidValue                                                           17.0
MonthlyReportNarcoticRemainValue                                                       3508.0
ReferCustomerName                                                โรงพยาบาลทั่วไปขนาดใหญ่นวเวช
CustomerAddressProvinceName                                 

In [216]:
dn_corr_req = wran_district3.loc[wran_district3['amp_code'].isna()][['fullAddress','CustomerAddressProvinceName', 'district_name','zip_code']].drop_duplicates()

In [217]:
dn_corr_req.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62 entries, 24796 to 286088
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   fullAddress                  62 non-null     object
 1   CustomerAddressProvinceName  62 non-null     object
 2   district_name                62 non-null     object
 3   zip_code                     62 non-null     object
dtypes: object(4)
memory usage: 2.4+ KB


In [218]:
dn_corr_req['dn2'] = np.nan

In [227]:
wran_district3['dn2'] = wran_district3['fullAddress'].str.extract(r'อำเภอ(.+?)\s') 

In [229]:
wran_district3.drop(['dn2'], axis=1, inplace=True)

In [230]:
wran_district3

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code
6,6,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,2024-03-16,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์,7205
31,31,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,2024-03-14,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้,8302
54,54,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2024-02-28,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง,7005
56,56,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
61,61,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288486,289469,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,2025-01-10,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส,9601
288487,289470,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,2025-01-10,2ae8194a-7022-4837-88b3-9baaab86a58e,d5ca07fe-e5b9-46b9-aeae-e2b3a69552d7,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae

In [236]:
dn_corr_req['dn2'] = dn_corr_req['fullAddress'].str.extract(r'อำเภอ(.+?)\s')

In [238]:
dn_corr_req['dn3'] = dn_corr_req['fullAddress'].str.extract(r'เขต(.+?)\s')

In [240]:
dn_corr_req['dn'] = dn_corr_req['dn2'].combine_first(dn_corr_req['dn3'])

In [242]:
dn_corr_req.loc[dn_corr_req['dn'].isna()]

,fullAddress,CustomerAddressProvinceName,district_name,zip_code,dn2,dn3,dn


<h3> push correction</h3>

In [225]:
dn_corr_req.iloc[1,0]

'10/87-88 หมู่ 4 ถนนกรุงเทพ-ปทุมธานี ตำบลบางขะแยง อำเภอเมืองปทุมธานี ปทุมธานี 10170'

In [222]:
dn_corr_req.iloc[2,0]

'21 โครงการแพลทินัมเพลส ชั้น 1 ห้องเลขที่ 1A-8 ถนนวัชรพล แขวงท่าแร้ง เขตบางเขน กรุงเทพมหานคร 10230'

In [206]:
wran_district3.loc[wran_district3['amp_code'].isna()][['CustomerAddressProvinceName']].drop_duplicates().iloc[0,0]

'กรุงเทพมหานคร'

In [ ]:
gdf.loc[gdf['pro_th'] == wran_district3.loc[wran_district3['amp_code'].isna()][['CustomerAddressProvinceName']].drop_duplicates().iloc[0,0]]

<h2>Feature to ampcode</h2>

In [120]:
corr_amp_df

,zipcode,amp_code,district_name
5411,10220,1005,บางเขน
7196,10330,1007,ปทุมวัน
7534,10240,1027,บึงกุ่ม
14271,10240,1027,บึงกุ่ม
15482,10500,1004,บางรัก
20806,10110,1033,คลองเตย
21094,12000,1301,เมืองปทุมธานี
22053,10260,1009,พระโขนง


<h3>Residual corrections</h3>

In [121]:
wran_district3.loc[wran_district3['amp_code'] =='nan'].iloc[0]

Unnamed: 0                                                                              24883
ReportDisburseNarcoticBeYear                                                           2567.0
ReportDisburseNarcoticMonth                                                               2.0
ProductTypeName                                               (วจ. 2) วัตถุออกฤทธิ์ในประเภท 2
MonthlyReportNarcoticName                              Alprazolam tablets 0.25 mg (Polipharm)
MonthlyReportNarcoticBalanceForwardValue                                               3525.0
MonthlyReportNarcoticReceiveValue                                                         0.0
MonthlyReportNarcoticPaidValue                                                           17.0
MonthlyReportNarcoticRemainValue                                                       3508.0
ReferCustomerName                                                โรงพยาบาลทั่วไปขนาดใหญ่นวเวช
CustomerAddressProvinceName                                 

In [123]:
pat = r'อำเภอ(.+?)\s|เขต(.+?)\s'

In [128]:
wran_district3.loc[wran_district3['amp_code'] =='nan'].iloc[0]['fullAddress']

'9 ถนนรัชดา-รามอินทรา แขวงนวลจันทร์ เขตบึงกุ่ม กรุงเทพมหานคร 10230'

In [129]:
def dn_extract(address):
    pat = r'อำเภอ(.+?)\s|เขต(.+?)\s'
    match = re.findall(pat, address)
    first_valid_match = next(s for s in match[0] if s)
    return first_valid_match

In [130]:
dn_extract(wran_district3.loc[wran_district3['amp_code'] =='nan'].iloc[0]['fullAddress'])

'บึงกุ่ม'

In [131]:
for i, v in wran_district3.loc[wran_district3['amp_code'] =='nan'].iterrows():
    fulladdress = v['fullAddress']
    dn = dn_extract(fulladdress)
    loc = corr_amp_df.loc[corr_amp_df['district_name'] == dn]
    if len(loc) >0:
        wran_district3.loc[i,'amp_code'] = loc.iloc[0]['amp_code']

In [145]:
# recast type
wran_district3['amp_code'] = wran_district3['amp_code'].astype(str)

In [146]:
wran_district3.loc[wran_district3['amp_code']=='nan']

,Unnamed: 0,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code


In [147]:
agg2_ct = wran_district3['amp_code'].value_counts().reset_index()

In [148]:
agg2_ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   amp_code  337 non-null    object
 1   count     337 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [88]:
gdf

,amp_code,amp_th,amp_en,pro_code,pro_th,pro_en,reg_nesdb,reg_royin,perimeter,area_sqkm,geometry
0,9015,สิงหนคร,Singhanakhon,90,สงขลา,Songkhla,South,South,72.908500,242.316953,"MULTIPOLYGON (((100.5598 7.25212, 100.57921 7...."
1,5307,พิชัย,Phichai,53,อุตรดิตถ์,Uttaradit,UpperNorth,North,142.230752,696.360661,"MULTIPOLYGON (((100.24898 17.25566, 100.2533 1..."
2,9016,คลองหอยโข่ง,Khlong Hoi Khong,90,สงขลา,Songkhla,South,South,85.990583,303.232680,"MULTIPOLYGON (((100.38452 6.82866, 100.37891 6..."
3,5308,ลับแล,Laplae,53,อุตรดิตถ์,Uttaradit,UpperNorth,North,126.116540,467.415328,"MULTIPOLYGON (((100.07131 17.59238, 100.07699 ..."
4,9013,ควนเนียง,Khuan Niang,90,สงขลา,Songkhla,South,South,75.657535,247.904319,"MULTIPOLYGON (((100.46056 7.18613, 100.45073 7..."
...,...,...,...,...,...,...,...,...,...,...,...
923,9009,รัตภูมิ,Rattaphum,90,สงขลา,Songkhla,South,South,129.224820,665.836293,"MULTIPOLYGON (((100.21558 6.99238, 100.19406 6..."
924,3109,พุทไธสง,Phutthaisong,31,บุรีรัมย์,Buriram,Northeast,Northeast,123.506957,311.768518,"MULTIPOLYGON (((103.02838 15.43731, 103.02172 ..."
925,1402,ท่าเรือ,Tha Ruea,14,พระนครศรีอยุธยา,Phra Nakhon Si Ayutthaya,Central,Central,62.464060,108.273098,"MULTIPOLYGON (((100.7665 14.52503, 100.74895 1..."
926,1403,นครหลวง,Nakhon Luang,14,พระนครศรีอยุธยา,Phra Nakhon Si Ayutthaya,Central,Central,57.527029,138.752678,"MULTIPOLYGON (((100.63178 14.40211, 100.61433 ..."


In [149]:
s_amp_th = agg2_ct.merge(
    gdf[['amp_code','amp_th']],
    how = 'left',
    validate = 'one_to_one',
    left_on = 'amp_code',
    right_on = 'amp_code'
)

In [151]:
s_amp_th.loc[s_amp_th['amp_th'].isna()]

,amp_code,count,amp_th


In [144]:
s_amp_th.loc[s_amp_th['amp_th'].isna()].iloc[0,0].astype(str)

np.str_('1027')

In [140]:
merge3.loc[merge3['district2'].str.contains('บึงกุ่ม')]

,data,zipcode,district,district2,amp_code,amp_th,amp_en,pro_code,pro_th,pro_en,reg_nesdb,reg_royin,perimeter,area_sqkm,geometry
337,ต.คลองกุ่ม อ.เขตบึงกุ่ม จ.กรุงเทพมหานคร 10240,10240,เขตบึงกุ่ม,บึงกุ่ม,1027,บึงกุ่ม,Bueng Kum,10,กรุงเทพมหานคร,Bangkok,Central,Central,23.123723,23.69003,"MULTIPOLYGON (((100.64971 13.84635, 100.64863 ..."
526,ต.คันนายาว อ.เขตบึงกุ่ม จ.กรุงเทพมหานคร 10240,10240,เขตบึงกุ่ม,บึงกุ่ม,1027,บึงกุ่ม,Bueng Kum,10,กรุงเทพมหานคร,Bangkok,Central,Central,23.123723,23.69003,"MULTIPOLYGON (((100.64971 13.84635, 100.64863 ..."
4799,ต.สะพานสูง อ.เขตบึงกุ่ม จ.กรุงเทพมหานคร 10240,10240,เขตบึงกุ่ม,บึงกุ่ม,1027,บึงกุ่ม,Bueng Kum,10,กรุงเทพมหานคร,Bangkok,Central,Central,23.123723,23.69003,"MULTIPOLYGON (((100.64971 13.84635, 100.64863 ..."


In [142]:
merge3.loc[merge3['amp_code'] == '1027']

,data,zipcode,district,district2,amp_code,amp_th,amp_en,pro_code,pro_th,pro_en,reg_nesdb,reg_royin,perimeter,area_sqkm,geometry
337,ต.คลองกุ่ม อ.เขตบึงกุ่ม จ.กรุงเทพมหานคร 10240,10240,เขตบึงกุ่ม,บึงกุ่ม,1027,บึงกุ่ม,Bueng Kum,10,กรุงเทพมหานคร,Bangkok,Central,Central,23.123723,23.69003,"MULTIPOLYGON (((100.64971 13.84635, 100.64863 ..."
526,ต.คันนายาว อ.เขตบึงกุ่ม จ.กรุงเทพมหานคร 10240,10240,เขตบึงกุ่ม,บึงกุ่ม,1027,บึงกุ่ม,Bueng Kum,10,กรุงเทพมหานคร,Bangkok,Central,Central,23.123723,23.69003,"MULTIPOLYGON (((100.64971 13.84635, 100.64863 ..."
4799,ต.สะพานสูง อ.เขตบึงกุ่ม จ.กรุงเทพมหานคร 10240,10240,เขตบึงกุ่ม,บึงกุ่ม,1027,บึงกุ่ม,Bueng Kum,10,กรุงเทพมหานคร,Bangkok,Central,Central,23.123723,23.69003,"MULTIPOLYGON (((100.64971 13.84635, 100.64863 ..."


In [162]:
fig = px.choropleth_map(
    s_amp_th,
    geojson = geojson_data,
    featureidkey="properties.amp_code",
    locations = 'amp_code',
    color = 'count',
    color_continuous_scale="Magenta",
    hover_name = 'amp_th',
)
fig.update_layout(
    map_center = {"lat": 13.736717, "lon": 100.523186}
)

fig.update_traces(
    marker_line_width=0.5,  # Set the border width (e.g., 1 pixel)
    marker = {
        "line": {
            "color": "#c2c6cc"
        }
    }
)
fig.write_html('C:/Users/ASUS/Downloads/s_amp_th_ct2025-09-25.html')
fig.show()

In [ ]:
# output pause on wran 3


In [164]:
wran_district3.to_csv('C:/Users/ASUS/Downloads/dist-rep-wran-ampcode-v4.csv')